In [169]:
import warnings
warnings.filterwarnings('ignore')

#Import Libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Read the file

In [170]:
house = pd.read_csv("train.csv")
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


EDA

In [171]:
house.shape
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

We see some missing values in the dataset.

In [172]:
(round(100*(house.isnull().sum()/len(house.index)),2).sort_values(ascending=False)).head(20)

PoolQC          99.52
MiscFeature     96.30
Alley           93.77
Fence           80.75
MasVnrType      59.73
FireplaceQu     47.26
LotFrontage     17.74
GarageYrBlt      5.55
GarageCond       5.55
GarageType       5.55
GarageFinish     5.55
GarageQual       5.55
BsmtFinType2     2.60
BsmtExposure     2.60
BsmtQual         2.53
BsmtCond         2.53
BsmtFinType1     2.53
MasVnrArea       0.55
Electrical       0.07
Id               0.00
dtype: float64

PoolQC, MiscFeature, Alley, Fence, MasVnrType, FireplaceQu have a lot of null data. We can drop them safely

In [173]:
house_data = house.drop(['FireplaceQu','MasVnrType','Fence','Alley','MiscFeature','PoolQC', 'Id'], axis=1)

In [174]:
house_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


Price values are very high compared to other fields. Let's log transform it.

In [175]:
house_data['SalePrice'] = np.log(house_data['SalePrice'])

In [176]:
(round(100*(house_data.isnull().sum()/len(house_data.index)),2).sort_values(ascending=False)).head(20)

LotFrontage     17.74
GarageYrBlt      5.55
GarageCond       5.55
GarageType       5.55
GarageFinish     5.55
GarageQual       5.55
BsmtFinType2     2.60
BsmtExposure     2.60
BsmtQual         2.53
BsmtFinType1     2.53
BsmtCond         2.53
MasVnrArea       0.55
Electrical       0.07
TotRmsAbvGrd     0.00
KitchenQual      0.00
KitchenAbvGr     0.00
BedroomAbvGr     0.00
HalfBath         0.00
FullBath         0.00
BsmtHalfBath     0.00
dtype: float64

From the data dictionary, we observe two variables are used to represent age of the building. So, we can derive that info into one.

In [177]:
house_data['Age'] = house_data['YrSold'] - house_data['YearBuilt']
house_data.drop(['YrSold', 'YearBuilt'], axis=1, inplace=True)
house_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,SaleType,SaleCondition,SalePrice,Age
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,WD,Normal,12.247694,5
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,WD,Normal,12.109011,31
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,WD,Normal,12.317167,7
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,WD,Abnorml,11.849398,91
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,WD,Normal,12.429216,8


Lets look into categorical variables

In [178]:
categorical = house_data.select_dtypes(include=['object'])
categorical.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [179]:
cat_col_names = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
       'SaleType', 'SaleCondition']
for i in cat_col_names:
    house_data[i].fillna("none", inplace=True)

Now, lets add dummy variables for categorical, add it to the data set and remove original columns.

In [180]:
dummy_cat = pd.get_dummies(categorical,drop_first=True)
house_data = pd.concat([house_data,dummy_cat],axis=1)
house_data.drop(cat_col_names,axis=1,inplace=True)
house_data.dropna(inplace=True)

# Split the data into test, train set.

In [181]:
df_train,df_test = train_test_split(house_data, train_size = 0.7, random_state = 100)
y_train=df_train.pop('SalePrice')
X_train=df_train
y_test=df_test.pop('SalePrice')
X_test=df_test

Now, using MinMaxScaler to scale the data.

In [182]:
scaler=MinMaxScaler()
numeric = X_train.dtypes[X_train.dtypes != "object"].index
X_train[numeric]=scaler.fit_transform(X_train[numeric])
X_test[numeric]=scaler.transform(X_test[numeric])


# Let's perform ridge regression now.

In [183]:
from sklearn.model_selection import GridSearchCV

In [184]:
params = {'alpha': [0.0001, 0.001, 0.01, 0.05,
                    0.1, 0.2, 0.3, 0.4, 0.5,
                    1.0, 2.0, 3.0, 4.0, 5.0,
                    6.0, 7.0, 8.0, 9.0, 10.0,
                    20, 50, 100, 500, 1000 ]}

ridge = Ridge()

# cross validation
folds = 5
model = GridSearchCV(estimator = ridge,
                        param_grid = params,
                        scoring= 'neg_mean_absolute_error',
                        cv = folds,
                        return_train_score=True,
                        verbose = 1)
model.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,
                                   8.0, 9.0, 10.0, 20, 50, 100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [185]:
ridge_cv_results = pd.DataFrame(model.cv_results_)
ridge_cv_results = ridge_cv_results[ridge_cv_results['param_alpha']<=500]
ridge_cv_results[['param_alpha', 'mean_train_score', 'mean_test_score', 'rank_test_score']].sort_values(by = ['rank_test_score'])


,param_alpha,mean_train_score,mean_test_score,rank_test_score
11,3.0,-0.071772,-0.093858,1
12,4.0,-0.073459,-0.093972,2
10,2.0,-0.069723,-0.094049,3
13,5.0,-0.074920,-0.094307,4
1,0.001,-0.054807,-0.094511,5
2,0.01,-0.055482,-0.094524,6
0,0.0001,-0.054760,-0.094553,7
14,6.0,-0.076268,-0.094755,8
9,1.0,-0.066824,-0.095024,9
15,7.0,-0.077527,-0.095288,10


Best alpha seems to be 3. Fit using that alpha.

In [194]:
alpha = 3
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
ridge.score(X_train,y_train)

y_train_pred = ridge.predict(X_train)
y_pred = ridge.predict(X_test)


In [195]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=100]
cv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.045436,0.010943,0.021761,0.005556,0.0001,{'alpha': 0.0001},-21620.177731,-18762.726421,-23334.835315,-18018.169480,...,-19919.387751,2179.325694,19,-12291.040747,-11731.301477,-11561.783568,-11851.521392,-11872.877826,-11861.705002,241.460417
1,0.061212,0.014622,0.026220,0.009455,0.001,{'alpha': 0.001},-21564.213170,-18694.725663,-23326.282874,-17945.598650,...,-19864.545335,2200.870941,16,-12282.767940,-11724.956076,-11562.205673,-11845.006005,-11876.179593,-11858.223058,239.216164
2,0.058175,0.015050,0.028359,0.007628,0.01,{'alpha': 0.01},-21234.799205,-18704.378041,-23256.214727,-17572.923383,...,-19647.250848,2257.705661,11,-12282.595537,-11846.945641,-11566.684201,-11910.423235,-11977.764102,-11916.882543,230.227857
3,0.060962,0.007402,0.029526,0.010226,0.05,{'alpha': 0.05},-21065.521762,-20296.410055,-23021.822524,-17718.469993,...,-19882.824114,2131.536394,18,-12723.791417,-12582.031267,-11587.258135,-12826.810821,-12711.735204,-12486.325369,456.204246
4,0.062053,0.013132,0.026648,0.006174,0.1,{'alpha': 0.1},-20940.994663,-21000.497358,-22791.800355,-18166.993518,...,-20087.070664,1953.382400,22,-13213.542928,-13012.158266,-11615.587126,-13396.560842,-13222.679020,-12892.105636,649.767326


In [196]:
from sklearn import metrics


print("R-Square of train data =", '%.2f' % r2_score(y_train, y_train_pred))
print("R-Square of test data =", '%.2f' % r2_score(y_test, y_pred))
print("MSE of train data =", '%.2f' % mean_squared_error(y_train, y_train_pred))
print("MSE of test data =", '%.2f' % mean_squared_error(y_test, y_pred))

R-Square of train data = 0.92
R-Square of test data = 0.88
MSE of train data = 0.01
MSE of test data = 0.02


# Lasso Regression

In [198]:
alphas = [0.0001, 0.001, 0.01, 0.05,
                    0.1, 0.2, 0.3, 0.4, 0.5,
                    1.0, 2.0, 3.0, 4.0, 5.0,
                    6.0, 7.0, 8.0, 9.0, 10.0,
                    20, 50, 100, 500, 1000 ]
lasso = LassoCV(alphas=alphas, cv=5)  # 5-fold cross-validation
lasso.fit(X_train, y_train)

LassoCV(alphas=[0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0,
                3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500,
                1000],
        cv=5)

In [199]:
lasso.alpha_

0.001

Best alpha for Lasso is 0.001

In [200]:
y_train_pred = lasso.predict(X_train)
y_pred = lasso.predict(X_test)

print("R-Square of train data =", '%.2f' % r2_score(y_train, y_train_pred))
print("R-Square of test data =", '%.2f' % r2_score(y_test, y_pred))
print("MSE of train data =", '%.2f' % mean_squared_error(y_train, y_train_pred))
print("MSE of test data =", '%.2f' % mean_squared_error(y_test, y_pred))

R-Square of train data = 0.90
R-Square of test data = 0.88
MSE of train data = 0.01
MSE of test data = 0.02


It is clear from the metrics that Lasso and Ridge performs very similar on the test data.